In [1]:
import numpy as np
import os
import glob
from astropy.io import fits
from astropy.table import Table, unique, Column, vstack
import matplotlib.pyplot as plt
import pandas as pd
from qn_analysis.desi_utils import *

### Merge the separate zbest files from redrock into a single file.

In [43]:
#rrdir = '/global/cfs/projectdirs/desi/users/jfarr/rrboss_dr12/runs/rr_v5_7_0/output/'
rrdir = '/global/cfs/cdirs/desi/spectro/redux/andes/tiles/68002/20200315/'
f_out = '/global/homes/j/jfarr/Projects/QuasarNET_DESI/rr_results/rr_SV0_68002_20200315.fits'

fi = glob.glob(rrdir+'/zbest*.fits')
print(len(fi))

10


In [44]:
## Concatenate output from all zbest files.
rr = fits.open(fi[0])
expid = int(fi[0].split('-')[-1][:-5])
rr_data = Table(rr[1].data)
expids = expid*np.ones(len(rr_data))
rr_data.add_column(Column(data=expids,name='EXPID',dtype='i8'))
rr.close()
if len(fi)>1:
    for i,f in enumerate(fi[1:]):
        rr = fits.open(f)
        expid = int(f.split('-')[-1][:-5])
        f_rr_data = Table(rr[1].data)
        expids = expid*np.ones(len(f_rr_data))
        f_rr_data.add_column(Column(data=expids,name='EXPID',dtype='i8'))
        rr_data = vstack([rr_data,f_rr_data])
        rr.close()
        print('Stacking zbest files: {:.2%}'.format((i+2)/len(fi)),end='\r')
print('')
print(len(rr_data))
rr_data.sort(['TARGETID','EXPID'])
rr_data[:5]

Stacking zbest files: 100.00%
5000


TARGETID,CHI2,COEFF [10],Z,ZERR,ZWARN,NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,NUMEXP,NUMTILE,EXPID
int64,float64,float64,float64,float64,int64,int64,str6,str20,int64,float64,int32,int32,int64
35185923930984962,11085.09177994728,120.51084008510408 .. -12.388555080372461,0.716799793437332,8.008771225229183e-05,0,7927,GALAXY,,10,462.9570941776037,0,1,20200315
35185923930985294,11203.254020661116,54.24157972394388 .. 1.132740341585059,0.804694385052438,9.692301971279603e-05,0,7927,GALAXY,,10,108.44522465765476,0,1,20200315
35185923930985431,11519.481780067106,2609.12229402862 .. 0.0,0.0001468951025535693,5.422923273618192e-06,0,7927,STAR,G,5,9436.423557981829,0,1,20200315
35185923930986052,8.999999999999996e+99,0.0 .. 0.0,-0.0019957014509158224,4.131159871613317e-48,1570,0,STAR,CV,3,1.942668892225729e+84,0,1,20200315
35185923935177997,11460.257465936244,174.50319654031534 .. -0.08255529289748459,0.6803965253708565,9.845858679498947e-05,0,7929,GALAXY,,10,928.4690854474902,0,1,20200315


In [46]:
rr_data.write(f_out,overwrite=True)

#### Open the qnAll file(s) we want, stitch them together and save.

In [50]:
fi_qn_loc = '/global/cfs/projectdirs/desi/users/jfarr/MiniSV_data/SV0/qn_export_output/coadd/'
fi_qn_form = 'qnAll-MiniSV_68002_20200315.fits'
fi_out = '/global/homes/j/jfarr/Projects/QuasarNET_DESI/qn_results/qn_SV0_68002_20200315.fits'

In [51]:
import glob
fi_qn = glob.glob(fi_qn_loc+fi_qn_form)
data = []
for f_qn in fi_qn:
    qn = fits.open(f_qn)
    w = (~(qn[1].data['IN_TRAIN'].astype('bool')))
    f_data = np.array(qn[1].data[w])
    data.append(f_data)
data = np.concatenate(data)
table = Table(data)
table.sort('THING_ID')
table[:5]

THING_ID,PLATE,MJD,FIBERID,ZBEST,IS_QSO,IN_TRAIN,C_LINES [6],Z_LINES [6],C_LINES_BAL,Z_LINES_BAL
int64,int32,int32,int32,float64,int64,int64,float64,float64,float64,float64
35185923935178324,68002,20200315,508,2.0022772952279952,1,0,1.0 .. 1.5746745677140336e-14,2.0022772952279952 .. -0.4265407152786532,0.9999984502792358,2.017358359966641
35185923939370065,68002,20200315,478,2.6565135073893535,0,0,2.073732616736379e-07 .. 1.801458830198721e-14,2.6343459033792027 .. -0.238498364762191,9.131026490472038e-10,2.6720305533237583
35185923939371431,68002,20200315,397,2.1530906113288966,1,0,0.9999986886978149 .. 1.577927198626039e-11,2.137075548380123 .. 0.3570473018378155,2.416289134998806e-06,2.1627778607711385
35185923939372655,68002,20200315,364,1.9671195752935873,0,0,7.720559551671613e-06 .. 2.031525836443393e-09,2.081454878129087 .. 0.4520716761253831,3.643215450210846e-06,2.13483162554244
35185923939373842,68002,20200315,394,1.8412341176825446,0,0,1.3036019312628877e-11 .. 4.219747899111103e-19,2.5504349938447386 .. -0.11977604245566265,9.147496484729345e-07,2.5296908729764525


In [52]:
table_unique = unique(table, keys=['PLATE', 'MJD', 'FIBERID'])
table = fits.BinTableHDU(table_unique)
prihdr = fits.Header()
prihdu = fits.PrimaryHDU(header=prihdr)
hdulist = fits.HDUList([prihdu, table])
hdulist.writeto(fi_out,overwrite=True)
hdulist.close()

### Convert a VI results csv file to a QN truth file.

In [35]:
f_vi = '/global/cfs/cdirs/desi/sv/vi/TruthTables/truth_table_QSO_v1.2.csv'
vi = pd.read_csv(f_vi, header=0)
print('INFO: {} VI results found'.format(len(vi)))
vi[:10]

INFO: 917 VI results found


,TARGETID,Redrock z,best z,best quality,Redrock spectype,best spectype,all VI issues,all VI comments,merger comment,N_VI,...,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z
0,35185929937227607,1.4995,1.49950,1.0,GALAXY,GALAXY,--,Poor noise subtraction Maybe OII correct but...,None,5,...,696,0.728820,0.730633,1.221301,0.566604,0.568014,0.949472,0.916734,0.943119,0.967773
1,35185929937228335,0.4102,0.41020,3.6,GALAXY,GALAXY,--,--,none,5,...,657,0.228894,0.745470,1.842765,0.178230,0.580468,1.434888,0.916849,0.943198,0.967819
2,35185929937229340,1.4944,1.49440,4.0,QSO,QSO,--,--,none,5,...,652,2.102620,4.026039,6.276346,1.635143,3.130926,4.880920,0.917295,0.943508,0.967997
3,35185929941418536,1.0804,1.08032,3.8,GALAXY,GALAXY,--,Both galaxy and QSO in spectrum,none,5,...,194,1.567732,2.329047,4.268086,1.221243,1.814298,3.324784,0.917957,0.943966,0.968260
4,35185929941418791,0.0002,0.00020,3.0,STAR,STAR,--,Large noise,None,5,...,228,0.839756,2.391115,4.388883,0.654225,1.862836,3.419229,0.918313,0.944213,0.968401
5,35185929941419181,0.4586,0.45860,1.0,GALAXY,GALAXY,--,Poor noise subtraction at 4400. Splice error 5...,None,5,...,53,0.239351,0.855025,1.875840,0.186145,0.664957,1.458849,0.916342,0.942847,0.967617
6,35185929941419381,0.9357,0.93570,4.0,GALAXY,QSO,C,--,None,5,...,211,4.135710,4.465388,7.363845,3.221942,3.478779,5.736834,0.917467,0.943627,0.968065
7,35185929941420031,1.5005,1.50050,1.0,GALAXY,GALAXY,--,--,None,5,...,94,0.483594,0.929053,1.420704,0.376337,0.722996,1.105602,0.918703,0.944483,0.968556
8,35185929941420850,0.0005,0.00050,4.0,STAR,STAR,--,--,None,5,...,248,1.739346,3.288057,4.919703,1.354533,2.560606,3.831267,0.917302,0.943513,0.967999
9,35185929941421006,0.0080,0.00834,2.0,GALAXY,STAR,--,--,Low z no big galaxy in image,5,...,232,0.924085,1.405844,1.309818,0.719449,1.094524,1.019763,0.917224,0.943459,0.967969


In [31]:
bal = [('bal' in comment.lower())|('broad absorption' in comment.lower()) for comment in vi['all VI comments']]

In [33]:
np.array(bal).sum()

41

In [34]:
(bal&(vi['best spectype']=='QSO')).sum()

20

In [30]:
(vi['best spectype']=='QSO').sum()

361

In [21]:
## Extract the information we want.
tids = vi['TARGETID']
z_vi = vi['best z']
class_vi = vi['best spectype']
secure = vi['best quality']>=2.5
possible = (vi['best quality']>0) & (~secure)

In [22]:
set(class_vi)

{'GALAXY', 'QSO', 'STAR'}

In [23]:
data = fits.open('/global/homes/j/jfarr/Projects/QuasarNET/data/MiniSV/SV0/1exp/MiniSV_data_1exp_00055626_SV0_QSO_lmax9800.fits')

In [24]:
## Make sure that all of our VI objects are in the data file.
w = np.in1d(tids,data[1].data['TARGETID'])
assert w.sum()==len(tids)

In [25]:
## Construct the dictionary needed for the truth file.
tr_dict = {}
tr_dict['TARGETID'] = tids
tr_dict['Z_VI'] = z_vi

tr_dict['SPID0'] = data[1].data['SPID0'][np.in1d(data[1].data['TARGETID'],tr_dict['TARGETID'])]
tr_dict['SPID1'] = data[1].data['SPID1'][np.in1d(data[1].data['TARGETID'],tr_dict['TARGETID'])]
tr_dict['SPID2'] = data[1].data['SPID2'][np.in1d(data[1].data['TARGETID'],tr_dict['TARGETID'])]

class_vi_coded = np.zeros(class_vi.shape)
class_vi_coded[class_vi=='QSO'] = 3
class_vi_coded[class_vi=='GALAXY'] = 4
class_vi_coded[class_vi=='STAR'] = 1
tr_dict['CLASS_PERSON'] = class_vi_coded

tr_dict['BAL_FLAG_VI'] = np.zeros(tr_dict['TARGETID'].shape)
tr_dict['BI_CIV'] = np.zeros(tr_dict['TARGETID'].shape)

confidences = np.zeros(secure.shape)
confidences[secure] = 3
confidences[possible] = 1
tr_dict['Z_CONF_PERSON'] = confidences.astype('int')

In [26]:
set(tr_dict['CLASS_PERSON'])

{1.0, 3.0, 4.0}

In [27]:
outname = '/global/homes/j/jfarr/Projects/QuasarNET/data/MiniSV/SV0//SV0_truth_tile68002_night20200315.fits'
table = Table(tr_dict)
table.write(outname, format='fits', overwrite=True)

### Combine BAL catalogues together.

In [18]:
fi_bal_loc = '/global/cfs/cdirs/desi/users/martini/SV0/68002/'
fi_bal_form = 'baltable-*-68002-20200315.fits'
fi_out = '/global/homes/j/jfarr/Projects/QuasarNET_DESI/balfinder_results/balfinder_SV0_68002_20200315.fits'

In [19]:
import glob
fi_bal = glob.glob(fi_bal_loc+fi_bal_form)
data = []
for f_bal in fi_bal:
    bal = fits.open(f_bal)
    f_data = np.array(bal[1].data)
    data.append(f_data)
data = np.concatenate(data)
table = Table(data)
table[:5]

TARGETID,TARGET_RA,TARGET_DEC,Z,ZERR,ZWARN,PCA_COEFFS [5],PCA_CHI2,BAL_PROB,BI_CIV,ERR_BI_CIV,NCIV_2000,VMIN_CIV_2000 [5],VMAX_CIV_2000 [5],POSMIN_CIV_2000 [5],FMIN_CIV_2000 [5],AI_CIV,ERR_AI_CIV,NCIV_450,VMIN_CIV_450 [17],VMAX_CIV_450 [17],POSMIN_CIV_450 [17],FMIN_CIV_450 [17],BI_SIIV,ERR_BI_SIIV,NSIIV_2000,VMIN_SIIV_2000 [5],VMAX_SIIV_2000 [5],POSMIN_SIIV_2000 [5],FMIN_SIIV_2000 [5],AI_SIIV,ERR_AI_SIIV,NSIIV_450,VMIN_SIIV_450 [17],VMAX_SIIV_450 [17],POSMIN_SIIV_450 [17],FMIN_SIIV_450 [17]
int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,float32
35185941920350460,200.71733,7.389993,1.8865774,0.00018747688,0.0,50.01192 .. -2.4200065,1.314995,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,687.1845,18.479765,3,17278.844 .. -1.0,17815.229 .. -1.0,17761.59 .. -1.0,-0.047162425 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0
35185941920351213,200.84016,7.438364,3.1780255,0.00028696412,0.0,25.421469 .. 0.301946,1.3252279,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,80.87148,7.758924,3,22446.428 .. -1.0,22965.246 .. -1.0,22742.896 .. -1.0,-0.22864194 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0
35185941920351477,200.7996,7.455114,1.9473405,0.00018505206,0.0,29.595219 .. 0.32942116,1.2865039,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0
35185941920352479,200.80397,7.5122623,2.3024974,0.00038030322,0.0,14.54834 .. 0.24404895,1.2991899,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0
35185941920352484,200.88284,7.513208,3.806303,0.00023492187,0.0,15.568284 .. 0.95778686,1.5597934,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0


In [20]:
table.write(fi_out, format='fits', overwrite=True)

### Make a directory structure with targetids in
For each (plate,mjd) pair, make a directory targetids/{plate}/, and create a file within it targetids-plate-mjd.txt

In [37]:
## Get all spectra for objects in DR12Q supserset
w = np.in1d(spall_dr12[1].data['THING_ID'],sdr12q[1].data['THING_ID'])
w &= (spall_dr12[1].data['THING_ID']!=-1)

In [39]:
## Get the set of (plate,mjd) pairs that cover these spectra
plate = spall_dr12[1].data['PLATE'][w].astype('i8')
mjd = spall_dr12[1].data['MJD'][w].astype('i8')
fiberid = spall_dr12[1].data['FIBERID'][w].astype('i8')

pm_set = set([(p,m) for p,m in zip(plate,mjd)])
print(len(pm_set))

2493


In [40]:
## Get the list of targetids for these spectra
targetids = platemjdfiber2targetid(plate,mjd,fiberid)

In [46]:
outpath = '/global/cfs/projectdirs/desi/users/jfarr/rrboss_dr12/'
for i,(p,m) in enumerate(pm_set):
    w = ((np.array(list(zip(plate,mjd)))==(p,m)).all(axis=1))
    pm_targetids = targetids[w]
    dirname = outpath+'/targetids/{}'.format(p)
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
    with open(dirname+'/targetids-{}-{}.txt'.format(p,m),'w') as text_file:
        for t in pm_targetids[:-1]:
            text_file.write('{},'.format(t))
        text_file.write('{}'.format(pm_targetids[-1]))
    print('{:.2%}'.format((i+1)/len(pm_set)),end='\r')

### Combine multiple SQUEzE outputs, removing duplicates in each file.

In [102]:
fi_sq = glob.glob('/project/projectdirs/desi/users/iprafols/SQUEzE/Mini-SV/68002/')
out_filename = '../sq_results/sq_SV0_00055628.fits'
print(len(fi_sq))

10


In [103]:
tables = []
for f_sq in fi_sq:
    sq = fits.open(f_sq)
    w = (~sq[1].data['DUPLICATED'])
    f_table = Table(sq[1].data[w])
    tables.append(f_table)
table = vstack(tables)
table.sort('TARGETID')
table[:5]

TARGETID,TARGET_RA,TARGET_DEC,CMX_TARGET,SPECID,LYB_RATIO,LYB_RATIO_SN,LYB_RATIO2,LYA_RATIO,LYA_RATIO_SN,LYA_RATIO2,SIIV_RATIO,SIIV_RATIO_SN,SIIV_RATIO2,CIV_RATIO,CIV_RATIO_SN,CIV_RATIO2,CIV_BLUE_RATIO,CIV_BLUE_RATIO_SN,CIV_BLUE_RATIO2,CIV_RED_RATIO,CIV_RED_RATIO_SN,CIV_RED_RATIO2,CIII_RATIO,CIII_RATIO_SN,CIII_RATIO2,NEIV_RATIO,NEIV_RATIO_SN,NEIV_RATIO2,MGII_RATIO,MGII_RATIO_SN,MGII_RATIO2,NEV_RATIO,NEV_RATIO_SN,NEV_RATIO2,OII_RATIO,OII_RATIO_SN,OII_RATIO2,HB_RATIO,HB_RATIO_SN,HB_RATIO2,OIII_RATIO,OIII_RATIO_SN,OIII_RATIO2,HA_RATIO,HA_RATIO_SN,HA_RATIO2,Z_TRY,PEAK_SIGNIFICANCE,ASSUMED_LINE,PROB_CLASS1,PROB_CLASS3,PROB_CLASS30,PROB_CLASS35,PROB_CLASS45,PROB_CLASS305,PROB_CLASS4,CLASS_PREDICTED,PROB,DUPLICATED
int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str15,float64,float64,float64,float64,float64,float64,float64,int64,float64,bool
35185923930984962,201.35337359935357,6.822608470758955,9007199254744064,35185923930984962,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.5819250405393256,-1.5830266435216216,0.20766097718733595,0.9407725119872348,-0.576219410651012,0.040108215755704314,0.8806976838225048,-1.032342902905168,0.11105520546150553,0.9479317416965869,-0.6755523535550004,0.04446361985042972,1.0961437322685887,3.409547881567021,0.010333974427315505,0.9902181967604592,-0.12758476985558515,0.010333974427315505,nan,nan,nan,0.8916177052356149,8.26252326317139,hb,0.27184817245193305,0.09727301895671307,0.008371360778606831,0.15744466789172587,0.25017811667940293,0.18032529523585225,0.03455936800576612,1,0.1056443797353199,False
35185923930985294,201.32750079573955,6.8426562871904375,1024,35185923930985294,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,none,nan,nan,nan,nan,nan,nan,nan,-1,nan,False
35185923930985431,201.2808097302484,6.846880974472636,8589935139,35185923930985431,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.0072277290067426,0.4947947935190738,0.010775657357436023,1.0156651922920361,1.7423515056361731,0.03258241942392968,1.0404004509364868,2.9797569028753017,0.0011272592199203667,1.0010543698885812,0.09121597220484155,0.014885175056405573,1.0239568940370138,2.661329816464819,0.08520091750333368,0.9633122711325609,-2.279706576688966,0.08520091750333368,nan,nan,nan,0.9297058176706552,11.94845438212471,hb,0.5074075786297046,0.08438045451559383,0.006971029902958535,0.0919263030831775,0.20694747304387293,0.08386778034232445,0.018499380482368478,1,0.09135148441855236,False
35185923930986052,201.2238856442244,6.864334846019875,4096,35185923930986052,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,none,nan,nan,nan,nan,nan,nan,nan,-1,nan,False
35185923935177997,201.5024208514255,6.777666203513355,9007199254742016,35185923935177997,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,none,nan,nan,nan,nan,nan,nan,nan,-1,nan,False


In [104]:
prihdr = fits.Header()
prihdu = fits.PrimaryHDU(header=prihdr)
table = fits.BinTableHDU(table)
hdulist = fits.HDUList([prihdu, table])
hdulist.writeto(out_filename,overwrite=True)
hdulist.close()

### Read the pipeline classes/redshifts from DR12 spall, and put all results into a simplified file.

In [65]:
w = np.in1d(spall_dr12[1].data['THING_ID'],sdr12q[1].data['THING_ID'])
w &= (spall_dr12[1].data['THING_ID']!=-1)
w.sum()

627751

In [66]:
plate = spall_dr12[1].data['PLATE'][w].astype('i8')
mjd = spall_dr12[1].data['MJD'][w].astype('i8')
fiberid = spall_dr12[1].data['FIBERID'][w].astype('i8')

targetid = platemjdfiber2targetid(plate,mjd,fiberid)
spectype = spall_dr12[1].data['CLASS'][w]
z = spall_dr12[1].data['Z'][w]
zwarn = spall_dr12[1].data['ZWARNING'][w]

thing_id = [pmf2tid_dr12[(p,m,f)] for p,m,f in zip(plate,mjd,fiberid)]

data_list = list(zip(targetid,spectype,z,zwarn,thing_id))
dtype = [('TARGETID','i8'),('SPECTYPE','U8'),('Z','f8'),('ZWARN','i8'),('THING_ID_DR12','i8')]

data = np.array(data_list,dtype=dtype)

In [67]:
table = Table(data)
table[:5]

TARGETID,SPECTYPE,Z,ZWARN,THING_ID_DR12
int64,str8,float64,int64,int64
3586551810016,QSO,2.2423312664031982,0,87897905
3586551810018,QSO,2.259599208831787,0,96906927
3586551810020,QSO,3.1812267303466797,0,96907939
3586551810032,STAR,-0.0031940187327563763,4,87896919
3586551810036,STAR,-0.00046957170707173645,0,87897024


In [68]:
table.write('../dr12pipe_results/dr12pipe_sdr12q.fits',overwrite=True)

### Do the same for DR14

In [82]:
w = np.in1d(spall_dr12[1].data['THING_ID'],sdr12q[1].data['THING_ID'])
w &= (spall_dr12[1].data['THING_ID']!=-1)
w.sum()

627751

In [83]:
targetid_sdr12q = platemjdfiber2targetid(spall_dr12[1].data['PLATE'][w],spall_dr12[1].data['MJD'][w],spall_dr12[1].data['FIBERID'][w])
targetid_dr14 = platemjdfiber2targetid(spall_dr14[1].data['PLATE'],spall_dr14[1].data['MJD'],spall_dr14[1].data['FIBERID'])
w = np.in1d(targetid_dr14,targetid_sdr12q)

In [85]:
plate = spall_dr14[1].data['PLATE'][w].astype('i8')
mjd = spall_dr14[1].data['MJD'][w].astype('i8')
fiberid = spall_dr14[1].data['FIBERID'][w].astype('i8')

targetid = platemjdfiber2targetid(plate,mjd,fiberid)
spectype = spall_dr14[1].data['CLASS'][w]
z = spall_dr14[1].data['Z'][w]
zwarn = spall_dr14[1].data['ZWARNING'][w]

thing_id = [pmf2tid_dr12[(p,m,f)] for p,m,f in zip(plate,mjd,fiberid)]

data_list = list(zip(targetid,spectype,z,zwarn,thing_id))
dtype = [('TARGETID','i8'),('SPECTYPE','U8'),('Z','f8'),('ZWARN','i8'),('THING_ID_DR12','i8')]

data = np.array(data_list,dtype=dtype)

In [86]:
table.write('../dr14pipe_results/dr14pipe_sdr12q.fits',overwrite=True)